In [1]:
#Install packages
!pip install pydriller

     |████████████████████████████████| 65 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 180 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 2.4 MB/s  eta 0:00:01
     |████████████████████████████████| 63 kB 1.4 MB/s  eta 0:00:01


In [2]:
#Import Repository
from pydriller import Repository
from datetime import datetime

In [71]:
#Extract github repo from known_apks
import re
import requests

with open('known_apks.txt', 'r') as f:
    githubApks = set()
    invalid = set()
    line = f.readline()
    while line:
        if "github" in line.lower():
            m = re.search("\w*\.\w*(?=_)", line)
            if m and "github" not in m.group():
                apk = m.group().split('.')
                url = 'https://github.com/{}/{}.git\n'.format(apk[0], apk[1])
                if url not in githubApks and url not in invalid:
                    try:
                        r = requests.get(url)
                        if r.status_code == 200:
                            print('✅ {}'.format(url))
                            githubApks.add(url)
                        else:
                            invalid.add(url)
                            print('❌ {}'.format(url))
                    except requests.ConnectionError:
                        print('Github error'.format(url))
        line = f.readline()
file = 'githubapk.txt'
if os.path.isfile(file):
    os.remove(file)
with open(file) as f2:
    f2.writelines(githubApks)

✅ https://github.com/aba2l/taswast.git

✅ https://github.com/alijc/cricketsalarm.git

✅ https://github.com/andremiras/etheroll.git

✅ https://github.com/andremiras/qrscan.git

❌ https://github.com/andremiras/zbarcamdemo.git

✅ https://github.com/ashutoshgngwr/noice.git

❌ https://github.com/ashutoshgngwr/tenbitclockwidget.git

❌ https://github.com/axet/audiorecorder.git

❌ https://github.com/axet/binauralbeats.git

❌ https://github.com/axet/bookreader.git

❌ https://github.com/axet/callrecorder.git

❌ https://github.com/axet/darknessimmunity.git

❌ https://github.com/axet/filemanager.git

❌ https://github.com/axet/hourlyreminder.git

❌ https://github.com/axet/maps.git

❌ https://github.com/axet/mover.git

❌ https://github.com/axet/smsgate.git

❌ https://github.com/axet/tonegenerator.git

❌ https://github.com/axet/torrentclient.git

❌ https://github.com/catfriend1/syncthingandroid.git

❌ https://github.com/characterdog/bmicalculator.git

❌ https://github.com/characterdog/share_my_number

In [93]:
#Extract github repo from repo and archive
import re
import requests
def parse_fdroid(path):
    with open('{}_parse'.format(path), 'x') as out:
        with open(path, 'r') as f:
            archive_apks = set()
            invalid = set()
            line = f.readline()
            while line:
                m = re.search("\w*\.\w*(?=_\d)", line)
                if m:
                    apk = m.group().split('.')
                    urlGithub = 'https://github.com/{}/{}.git\n'.format(apk[0], apk[1])
                    if urlGithub not in archive_apks and urlGithub not in invalid:
                        try:
                            r = requests.get(urlGithub)
                            if r.status_code == 200:
                                #print('✅ {}'.format(urlGithub))
                                archive_apks.add(urlGithub)                                
                                out.write(urlGithub)
                            else:
                                invalid.add(urlGithub)
                                #print('❌ {}'.format(urlGithub))
                        except requests.ConnectionError:
                            print('Github error'.format(urlGithub))
                    urlGitlab = 'https://gitlab.com/{}/{}.git\n'.format(apk[0], apk[1])
                    if urlGitlab not in archive_apks and urlGitlab not in invalid:
                        try:
                            r = requests.get(urlGitlab)
                            if r.status_code == 200:
                                #print('✅ {}'.format(urlGitlab))
                                archive_apks.add(urlGitlab)
                                out.write(urlGitlab)
                            else:
                                invalid.add(urlGitlab)
                                #print('❌ {}'.format(urlGitlab))
                        except requests.ConnectionError:
                            print('Gitlab error'.format(urlGitlab))
                line = f.readline()

In [94]:
parse_fdroid('fdroid_repo')
parse_fdroid('fdroid_archive')

In [51]:
#Extracts repo url
with open('repo.txt') as f:
    url = [x.rstrip() for x in f.readlines()]

In [4]:
#Extract project name from project url
def get_project_name(repo_url):
    return os.path.basename(repo_url[:-4])

In [23]:
import subprocess
import os
import shutil

def extract_commits(repo_url, clean_up):
    start_time = datetime.now()
    project_info = dict()
    project_name = get_project_name(repo_url)
    project_dir = "repo/{}".format(project_name)
    #Clone repo if needed
    print("Cloning repository {}".format(project_name))
    if not os.path.isdir(project_dir):
        subprocess.run(["git","clone", repo_url, project_dir])
    
    print("Creating repository object {}".format(project_name))
    repo = Repository(project_dir, 
                      only_modifications_with_file_types = ".java",
                      since=datetime(2021, 1, 1))
    
    print("Start commit analysis")
    for commit in repo.traverse_commits():
        commit_diff = dict()
        for modified_file in commit.modified_files:
            #No added modifications
            if not modified_file.filename.endswith(".java") or not modified_file.diff_parsed['added']:
                continue
            
            #Store file modfications
            commit_diff[modified_file.filename] = modified_file.diff_parsed['added']
        #Store commit modifications
        if commit_diff:
            project_info[str(commit.hash)] = commit_diff
    if clean_up:
        shutil.rmtree(project_dir)
    print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))
    return project_info


        

In [24]:
def extract_projects(url, clean_up = False):
    projects = dict()
    for repo_url in url:
        project_name = get_project_name(repo_url)
        projects[project_name] = extract_commits(repo_url,clean_up)
        print("{} analysis complete".format(project_name))
    return projects

In [25]:
import json
p = extract_projects(url, False)
file = 'data.json'
os.remove(file)
with open(file, "w") as f:
  json.dump(p, f, indent = 6)


Cloning repository Java


Cloning into 'repo/Java'...


Creating repository object Java
Start commit analysis
Time elapsed (hh:mm:ss.ms) 0:00:02.980953
Java analysis complete
Cloning repository elasticsearch
Creating repository object elasticsearch
Start commit analysis


Exception ignored in: <function Git.CatFileContentStream.__del__ at 0x7f983daddee0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/git/cmd.py", line 609, in __del__
    self._stream.read(bytes_left + 1)
ValueError: read of closed file


KeyboardInterrupt: 

# Pipeline 3.1 du paper
## Obtenir les repos
* Sortir une liste des repo git

## PyDriller
* Cloner
* Extraire un certains nombre de commit
* On veut la structure suivante: Repo -> commit -> file
* Utiliser TestFileDetector pour valider que le fichier est un test
* Associer le fichier de test au fichier de production avec TestFileMapping
* Conserver les changements dans un JSON